In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_excel('reservations.xlsx', 
                   names=['doc', 'spec', 'source', 'patient', 'date_done', 
                          'user', 'date_reserv', 'date_call', 'waiting_days'])

In [ ]:
df['date_done'] = pd.to_datetime(df['date_done'])
df['date_reserv'] = pd.to_datetime(df['date_reserv'])

In [ ]:
df.set_index('date_done', drop=False, inplace=True)

In [ ]:
# Поиск чаще всего записывающихся/записываемых

df \
    .groupby('patient', as_index=False) \
    .agg({'date_done': 'count'}) \
    .sort_values('date_done', ascending=False) \
    .head(30)

In [ ]:
# Поиск чаще всего записывающихся/записываемых в резрезе врача и источника записи

df \
    .groupby(['patient', 'doc', 'source'], as_index=False) \
    .agg({'date_done': 'count'}) \
    .sort_values('date_done', ascending=False) \
    .query('date_done > 10')

In [ ]:
# Максимальное число отмен и время последней отмены

df[df['date_call'] == 'Запись отменена'] \
    .groupby('doc') \
    .agg({'date_call': 'count', 'date_done': 'max'}) \
    .sort_values('date_call', ascending=False).head(30)

In [ ]:
# Создадим список нерабочих дней и рабочих суббот

holidays = ['2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',
            '2022-01-07', '2022-02-23', '2022-03-07', '2022-03-08', '2022-05-02',
            '2022-05-03', '2022-05-09', '2022-05-10', '2022-06-13', '2022-11-04']
working_saturday = ['2022-03-05']

In [ ]:
# Отметим рабочий или нерабочий день

df['is_business_day'] = df['date_reserv'].isin(working_saturday) | \
                                            (~(df['date_reserv'].isin(holidays)) & 
                                             ~(df['date_reserv'].dt.dayofweek >= 5))

In [ ]:
# Отметим тех, кто в течение периода посетил врача хотя бы раз

exclude = ['Запись отменена', 'Запись перенесена', 'Неявка']
approved_patient = df.query('date_call not in @exclude').patient.unique()
df['approved'] = df.patient.isin(approved_patient)

In [ ]:
# Выделим записи вне расписания

df['over_rasp'] = df.date_reserv.dt.hour == 0

In [ ]:
date_begin = '2022-01-01'

df_cancelled = df[df['date_call'] == 'Запись отменена'] \
                    .query('date_reserv >= @date_begin')

# Отмен по источникам
df_cancelled_by_src = df_cancelled \
                            .groupby(['doc', 'source'], as_index=False) \
                            .agg({'patient': 'count'}) \
                            .pivot(index='doc', columns='source', values='patient') \
                            .fillna(0).astype(int)

# Отмен по специалисту
df_cancelled_by_doc = df_cancelled \
                        .groupby('doc') \
                        .agg({'date_call': 'count'}) \
                        .sort_values('date_call', ascending=False)

df_by_date = df \
                .query('date_reserv >= @date_begin')

# Плановая запись на рабочие дни (без вне расписания)
df_reservations = df_by_date \
                    .query('date_call not in @exclude & is_business_day & not over_rasp') \
                    .groupby(['doc', 'source'], as_index=False) \
                    .agg({'date_reserv': 'count'}) \
                    .pivot(index='doc', columns='source', values='date_reserv') \
                    .fillna(0).astype(int)

# Статистика по отменам через интернет и регистратуру
df_cancel_stat = df \
                    .query('date_reserv >= @date_begin') \
                    .groupby('doc').agg({'date_reserv': 'nunique'}) \
                    .merge(df_cancelled_by_doc, left_index=True, right_index=True) \
                    .merge(df_cancelled_by_src, left_index=True, right_index=True) \
                    .assign(cancel_int_rate = lambda x: round((x.Интернет/x.date_reserv)*100, 1),
                            cancel_reg_rate = lambda x: round((x.Регистратура/x.date_reserv)*100, 1),
                            cancel_rate = lambda x: round((x.date_call/x.date_reserv)*100, 1)) \
                    .sort_values('cancel_reg_rate', ascending=False)

df_reservations \
    .merge(df_by_date
                .query('date_call not in @exclude & is_business_day & over_rasp')
                .groupby('doc')
                .agg({'date_reserv': 'count'}),
           left_index=True,
           right_index=True) \
    .merge(df_cancel_stat[['date_reserv', 'cancel_reg_rate']],
           left_index=True,
           right_index=True) \
    .sort_values('cancel_reg_rate', ascending=False) \
    .rename(columns={'date_reserv_x': 'Вне расписания', 'date_reserv_y': 'Всего талонов',
                     'cancel_reg_rate': 'Отмен регистратурой, %'})

In [ ]:
# Статистика по отменам

df_cancel_stat = df \
                    .groupby('doc').agg({'date_reserv': 'nunique'}) \
                    .merge(df_cancelled_by_doc, left_index=True, right_index=True) \
                    .merge(df_cancelled_by_src, left_index=True, right_index=True) \
                    .assign(cancel_int_rate = lambda x: round((x.Интернет/x.date_reserv)*100, 1),
                            cancel_reg_rate = lambda x: round((x.Регистратура/x.date_reserv)*100, 1),
                            cancel_rate = lambda x: round((x.date_call/x.date_reserv)*100, 1)) \
                    .rename(columns={'date_reserv': 'Записей', 'date_call': 'Талонов',
                                     'cancel_int_rate': 'Отмен через интернет, %',
                                     'cancel_reg_rate': 'Отмен через регистратуру, %',
                                     'cancel_rate': 'Всего отмен, %',}) \
                    .sort_values('Отмен через регистратуру, %', ascending=False)
df_cancel_stat

In [ ]:
df \
    .query('date_call not in @exclude') \
    .groupby('doc', as_index=False) \
    .agg({'date_reserv': 'nunique'})

In [ ]:
df_cancelled \
    .groupby([df_cancelled['date_done'].dt.month, df_cancelled['doc']]) \
    .agg({'date_call': 'count'}) \
    .reset_index() \
    .to_excel('cancelled.xlsx')

In [ ]:
df_cancelled.hist(by='doc', column='waiting_days', figsize=(20,10))

In [ ]:
sns.histplot(data=df_cancelled, x='waiting_days')

In [ ]:
df_cancelled \
    .groupby(['doc', 'source']) \
    .agg({'date_call': 'count'}) \
    .sort_values('date_call', ascending=False).head(30)

In [ ]:
df.query('date_call != "Запись отменена"')

In [ ]:
df.groupby(['patient', 'date_call']).agg({'date_reserv': 'count', 'waiting_days': 'sum'})

In [ ]:
df \
    .query('not approved & source == "Регистратура"') \
    .groupby(['patient', 'doc'], as_index=False) \
    .agg({'date_reserv': 'count', 'waiting_days': 'sum'}) \
    .sort_values('date_reserv', ascending=False) \
    .groupby('patient') \
    .agg({'doc': ['count', 'nunique']}) \
    .sort_values(('doc', 'count'), ascending=False)

In [ ]:
df \
    .query('not approved & source == "Интернет"') \
    .groupby(['patient', 'doc'], as_index=False) \
    .agg({'date_reserv': 'count', 'waiting_days': 'sum'}) \
    .sort_values('date_reserv', ascending=False)